### This exercise is to build logistic regression using Neural network, Build cat vs dog classifier

In [1]:
#impoet necessary libraires
import numpy as np
# import math
import cv2
import os
import glob
import random
np.random.seed(0)

In [2]:
# define custom variables
cat = 0
dog = 1
height = 130
width = 130

cat_train_filepath = 'data/cat_dog_ds/train/cats/*'
dog_train_filepath = 'data/cat_dog_ds/train/dogs/*'
cat_test_filepath = 'data/cat_dog_ds/test/cats/*'
dog_test_filepath = 'data/cat_dog_ds/test/dogs/*'

#define W and b metrices
# W shape would be (1,height*width*3)
# b would be scaler
# lr is learning rate

W = np.random.rand(height*width*3,1)
b = np.random.rand(1)
b
lr = 0.3

In [3]:
#define function which read train cats and dog dataset and create training dataset with labels

In [4]:
#Check the min dimension of the image
def create_x_y_set(filepath,height,width,label):
    down_width = width
    down_height = height
    down_points = (down_width, down_height)
    pics = glob.glob(filepath)
    train_images = []
    for pic in pics:
        img = cv2.imread(pic, cv2.IMREAD_COLOR)
        resized_down = cv2.resize(img, down_points, interpolation= cv2.INTER_LINEAR)
        cat_img = resized_down.reshape(height*width*3,1)/255
        train_images.append(cat_img)
    train_images = np.array(train_images).reshape(height*width*3,len(pics))
    train_label = np.array([label] * len(pics)).reshape(1,len(pics))
    return train_images, train_label

#define sigmoid function
def sigmoid_fun(x):
    s = 1/(1+ np.exp(-x))
    return s

# Sigmoid gradient
def sigmoid_grad(x):
    s = sigmoid_fun(x)
    ds = s*(1-s)
    return ds

# Implement L1 & L2 loss function
# L1(y,y_hat) = summation (i to m) abs(y(i) - y_hat(i))

def l1_loss(yhat, y):
    loss = sum(abs(yhat - y))
    return loss

def l2_loss(yhat, y):
    loss = sum((yhat-y)**2)
    return loss

def l2_loss_new(yhat, y):
    x = yhat-y
    loss = np.dot(x,x)
    return loss

def binary_entropy_loss(yhat, y):
    m = y.shape[1]
    yhat = np.clip(yhat, 1e-7, 1 - 1e-7)
    loss = -np.mean(((y*np.log(yhat))+(1-y)*np.log(1-yhat)))
    # loss = (np.dot(y,np.log(yhat))+np.dot(1-y,np.log(1-yhat)))
    return loss

# da is nothing but derivative of loss with respect to a = sigmoid(z), z = W.T*X+B
def partial_derivative_da(yhat,y):
    da = (-y/yhat) + (1-y)/(1-yhat)
    return da

def partial_derivative_dz(yhat,y):
    dz = (yhat-y)
    return dz

def partial_derivative_dw(dz,X):
    # print(f' dz shape - {dz.shape}')
    # print(f' X shape - {X.shape}')
    # print(f' X shape(1) - {X.shape[1]}')
    dw = np.dot(X,dz.transpose())/X.shape[1]
    # print(f' dw shape - {dw.shape}')
    return dw

def partial_derivative_db(dz):
    db = np.sum(dz)/dz.shape[1]
    return db

#forward pass
# Z = W.T*X+b

def calculate_yhat(X,W,b):
    z = np.dot(W.transpose(),X)+b
    yhat = sigmoid_fun(z)
    return yhat

def prediction(X_test,y_test,W,b):
    yhat = calculate_yhat(X_test,W,b)
    yhat = yhat.reshape(1, y_test.shape[1])
    correct_prediction = round(((y_test==yhat).sum()/y_test.shape[1])*100,2)
    print(f'Accuracy in test data {correct_prediction} %')


def create_train_test_data(cat_train_filepath,dog_train_filepath,cat_test_filepath,dog_test_filepath):
    cat_X_train, cat_y_train = create_x_y_set(cat_train_filepath,height,width,cat)
    dog_X_train, dog_y_train = create_x_y_set(dog_train_filepath,height,width,dog)
    cat_X_test, cat_y_test = create_x_y_set(cat_test_filepath,height,width,cat)
    dog_X_test, dog_y_test = create_x_y_set(dog_test_filepath,height,width,dog)
    X_train = np.concatenate([cat_X_train,dog_X_train],axis=1)
    y_train = np.concatenate([cat_y_train,dog_y_train],axis=1)
    X_test = np.concatenate([cat_X_test,dog_X_train],axis=1)
    y_test = np.concatenate([cat_y_test,dog_y_train],axis=1)
    return X_train, X_test, y_train, y_test

def train_model(cat_train_filepath,dog_train_filepath,cat_test_filepath,dog_test_filepath,W,b,lr):
    X_train, X_test, y_train, y_test = create_train_test_data(cat_train_filepath,dog_train_filepath,cat_test_filepath,dog_test_filepath)
    for i in range(100):
        yhat = calculate_yhat(X_train,W,b)
        yhat = yhat.reshape(1, y_train.shape[1])
        dz = partial_derivative_dz(yhat,y_train)
        dw = partial_derivative_dw(dz,X_train)
        db = partial_derivative_db(dz)
        # adjust the weights
        W = W - lr*dw
        b = b - lr*db
        bin_loss = binary_entropy_loss(yhat, y_train)
        correct_prediction = round(((y_train==yhat).sum()/y_train.shape[1])*100,2)
        print(f'epoch no - {i} binary_entropy_loss - {bin_loss} correct prediction - {correct_prediction} %')
    return W,b

def make_prediction(cat_train_filepath,dog_train_filepath,cat_test_filepath,dog_test_filepath,W,b):
    X_train, X_test, y_train, y_test = create_train_test_data(cat_train_filepath,dog_train_filepath,cat_test_filepath,dog_test_filepath)
    yhat = calculate_yhat(X_test,W,b)
    yhat = yhat.reshape(1, y_test.shape[1])
    correct_prediction = round(((y_test==yhat).sum()/y_test.shape[1])*100,2)
    print(f'Test dataset accuracy- {correct_prediction} %')

In [5]:

W,b = train_model(cat_train_filepath,dog_train_filepath,cat_test_filepath,dog_test_filepath,W,b,lr)
make_prediction(cat_train_filepath,dog_train_filepath,cat_test_filepath,dog_test_filepath,W,b)


epoch no - 0 binary_entropy_loss - 8.073516543203278 correct prediction - 49.91 %
epoch no - 1 binary_entropy_loss - 8.073516543203278 correct prediction - 49.91 %
epoch no - 2 binary_entropy_loss - 8.073516543203278 correct prediction - 49.91 %
epoch no - 3 binary_entropy_loss - 8.073516543203278 correct prediction - 49.91 %
epoch no - 4 binary_entropy_loss - 8.073516543203278 correct prediction - 49.91 %
epoch no - 5 binary_entropy_loss - 8.073516543203278 correct prediction - 49.91 %
epoch no - 6 binary_entropy_loss - 1.0000000494736474e-07 correct prediction - 100.0 %
epoch no - 7 binary_entropy_loss - 1.0000000494736474e-07 correct prediction - 100.0 %
epoch no - 8 binary_entropy_loss - 1.0000000494736474e-07 correct prediction - 100.0 %
epoch no - 9 binary_entropy_loss - 1.0000000494736474e-07 correct prediction - 100.0 %
epoch no - 10 binary_entropy_loss - 1.0000000494736474e-07 correct prediction - 100.0 %
epoch no - 11 binary_entropy_loss - 1.0000000494736474e-07 correct predi

/var/folders/jb/_bs3cjyn0wx77s6vhtrjwn1m0000gn/T/ipykernel_53614/2905522531.py:19: RuntimeWarning: overflow encountered in exp
  s = 1/(1+ np.exp(-x))


epoch no - 13 binary_entropy_loss - 1.0000000494736474e-07 correct prediction - 100.0 %
epoch no - 14 binary_entropy_loss - 1.0000000494736474e-07 correct prediction - 100.0 %
epoch no - 15 binary_entropy_loss - 1.0000000494736474e-07 correct prediction - 100.0 %
epoch no - 16 binary_entropy_loss - 1.0000000494736474e-07 correct prediction - 100.0 %
epoch no - 17 binary_entropy_loss - 1.0000000494736474e-07 correct prediction - 100.0 %
epoch no - 18 binary_entropy_loss - 1.0000000494736474e-07 correct prediction - 100.0 %
epoch no - 19 binary_entropy_loss - 1.0000000494736474e-07 correct prediction - 100.0 %
epoch no - 20 binary_entropy_loss - 1.0000000494736474e-07 correct prediction - 100.0 %
epoch no - 21 binary_entropy_loss - 1.0000000494736474e-07 correct prediction - 100.0 %
epoch no - 22 binary_entropy_loss - 1.0000000494736474e-07 correct prediction - 100.0 %
epoch no - 23 binary_entropy_loss - 1.0000000494736474e-07 correct prediction - 100.0 %
epoch no - 24 binary_entropy_los